### 01. Pseudonymization

In this notebook, we'll explore pseudonymization methods such as hashing, masking and format-preserving encryption.

For more reading on the topic, please see: 

- [Medium (Alex Ewerlöf): Anonymization vs. Pseudonymization](https://medium.com/@alexewerlof/gdpr-pseudonymization-techniques-62f7b3b46a56)
- [KIProtect: GDPR for Data Science](https://kiprotect.com/blog/gdpr_for_data_science.html)
- [IAPP: Anonymization and Pseudonymization Compared in relation to GDPR compliance](https://iapp.org/media/pdf/resource_center/PA_WP2-Anonymous-pseudonymous-comparison.pdf)

In [ ]:
import base64
from hashlib import blake2b

import pandas as pd
import json
import requests

from faker import Faker
from ff3 import FF3Cipher

#### Precheck: What is our data? 
- What information is contained in our data?
- What privacy concerns are there?
- How should we proceed?

In [ ]:
df = pd.read_csv('data/iot_example.csv')

In [ ]:
df.head()

#### Section One: Hashing

- Applying the blake2b hash
- Allowing for de-pseudonymization
- Creating a reusable method for hashing

In [ ]:
username = df.iloc[0,1]

In [ ]:
username

In [ ]:
hasher = blake2b()
hasher.update(username)
hasher.hexdigest()

Oops. What went wrong? How can we fix?

In [ ]:
# %load solutions/proper_encoding.py


Great! Now we have a hash. Michael is safe! (or [is he?](https://nakedsecurity.sophos.com/2014/06/24/new-york-city-makes-a-hash-of-taxi-driver-data-disclosure/))

But... what if we need to later determine that michaelsmith is a2a858011c091715....

In [ ]:
hasher.

Okay, let's try something that we can reverse...

In [ ]:
# From https://stackoverflow.com/questions/2490334/simple-way-to-encode-a-string-according-to-a-password

def encode(key, clear):
    enc = []
    for i in range(len(clear)):
        key_c = key[i % len(key)]
        #print(key_c)
        enc_c = (ord(clear[i]) + ord(key_c)) % 256
        #print(enc_c)
        enc.append(enc_c)
    return base64.urlsafe_b64encode(bytes(enc))

def decode(key, enc):
    dec = []
    enc = base64.urlsafe_b64decode(enc)
    for i in range(len(enc)):
        key_c = key[i % len(key)]
        dec_c = chr((256 + enc[i] - ord(key_c)) % 256)
        dec.append(dec_c)
    return "".join(dec)

In [ ]:
encode('supa_secret', username)

In [ ]:
decode('supa_secret', b'4N7TycDY0dbfzujb')

#### Challenge

- Can you come up with another string which will properly decode the secret which is *not* the same as the original key?
- Hint: Take a look at the encode method and use the print statements for a clue.

In [ ]:
# %load solutions/lockpick.py


Welp. That maybe is not so great... 

#### Section Two: Data Masking and Tokenization

- What should we mask?
- How?
- What do we do if we need realistic values?

In [ ]:
df.sample(2)

In [ ]:
super_masked = df.applymap(lambda x: 'NOPE')

In [ ]:
super_masked.head()

😜

Okay, no more jokes. But masking usually is just that. Replace your senstive data with some sort of represetation.

But instead, we could also tokenize the data. This means to replace it with random fictitious data. How do we tokenize this?

In [ ]:
fakes = Faker()

In [ ]:
fakes.name()

In [ ]:
fakes.

In [ ]:
fakes.user_name()

#### Challenge

Make a new column `pseudonym` which masks the data using the faker `user_name` method.

In [ ]:
# %load solutions/masked_pseudonym.py



Whaaaa!?!? Pretty cool, eh? 

(In case you want to read up on [how it works](https://github.com/joke2k/faker/blob/06d323f6cff95103d4ccda03f5d4ab2c45334e46/faker/providers/internet/__init__.py#L162))

But.. we can't reverse it. It is tuned per locale (usually using probabilities based on names in locale). That said, works fabulous for test data!

#### Step Three: Format-Preserving Encryption

In [ ]:
key = "2DE79D232DF5585D68CE47882AE256D6"
tweak = "CBD09280979564"

c6 = FF3Cipher.withCustomAlphabet(key, tweak, "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789_")

plaintext = "michaelsmith"
ciphertext = c6.encrypt(plaintext)

ciphertext

In [ ]:
decrypted = c6.decrypt(ciphertext)
decrypted

In [ ]:
df['username'] = df['username'].map(c6.encrypt)

Oh no! What does this message mean and how can we fix it?

In [ ]:
# %load solutions/pad_text.py


In [ ]:
def add_padding_and_encrypt(cipher, username):
    # add code here

In [ ]:
df['username'] = df['username'].map(lambda x: add_padding_and_encrypt(c6, x))

In [ ]:
df['username']

### Questions

- What would happen if someone found our key?
- What happens if a username ends in X?
- What properties do we need in our data in order to maintain encryption-level security?

#### Additional Challenge

How would we build our own format-preserving encryption?

In [ ]:
num_cipher = FF3Cipher.withCustomAlphabet(key, tweak, "0123456789")

In [ ]:
example = "2017-01-01T12:00:23"

In [ ]:
enc_date = num_cipher.encrypt(example.replace("T","").replace(":","").replace("-",""))

In [ ]:
enc_ts = f"{enc_date[:4]}-{enc_date[4:6]}-{enc_date[6:8]}T{enc_date[8:10]}:{enc_date[10:12]}:{enc_date[12:14]}"

In [ ]:
enc_ts

#### Homework Challenge

Create a function to format preserve another column in the data.

Return a new dataframe of just the pseudonymized data.